In [ ]:
import tensorflow as tf
import api.capital_features_api as cf
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, InputLayer, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
tf.config.list_physical_devices(device_type='GPU')

In [ ]:
train_data = cf.get_train_data(cf.prod_train_data_url, train_config_id=41146)
validation_data = cf.get_train_data(cf.prod_train_data_url, train_config_id=41147)

In [ ]:
X_train = np.array(train_data['featuresT'])
X_val = np.array(validation_data['featuresT'])

In [ ]:
# 获取形状信息
num_samples, num_timesteps, num_features = X_train.shape
# 将三维特征数组重塑为二维
X_train_reshaped = X_train.reshape(-1, num_features)
# 使用训练集的数据来拟合scaler
scaler.fit(X_train_reshaped)

# 标准化训练数据
X_train_scaled = scaler.transform(X_train_reshaped).reshape(num_samples, num_timesteps, num_features)
# 同样的，将验证集（如果有的话）重塑并转换
num_samples_val, num_timesteps_val, num_features_val = X_val.shape
X_val_reshaped = X_val.reshape(-1, num_features_val)
X_val_scaled = scaler.transform(X_val_reshaped).reshape(num_samples_val, num_timesteps_val, num_features_val)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, train_data['upSignals']))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val_scaled, validation_data['upSignals']))
BATCH_SIZE = 64  # 你可以根据需要调整这个值
train_dataset = train_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [ ]:
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data['upSignals']),
    y=train_data['upSignals']
)
class_weights = dict(zip(np.unique(train_data['upSignals']), weights))

In [ ]:
model = Sequential([
    tf.keras.layers.InputLayer(input_shape=(6, 71)),
    GRU(16, return_sequences=True),  # 减小GRU单元数目
    Dropout(0.2),  # 略微增加Dropout比例
    BatchNormalization(),
    GRU(8),  # 进一步减少RNN层大小
    Dropout(0.2),
    BatchNormalization(),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),  # 增加L2正则化
    Dropout(0.2),  # 继续使用较高的Dropout比例
    Dense(1, activation='sigmoid'),
])

model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall']
)


early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=20,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.5,
    patience=4,
    min_lr=1e-5,
    verbose=1
)

In [ ]:
his_up = model.fit(train_dataset, epochs=100, validation_data=validation_dataset, verbose=1,
                   callbacks=[early_stopping, reduce_lr], class_weight=class_weights)

In [ ]:
plt.plot(his_up.history['accuracy'], label='Training_UP')
plt.plot(his_up.history['val_accuracy'], label='Validation_UP')
plt.legend()
plt.show()